In [ ]:
from flask import Flask, request, redirect, render_template, flash, url_for
from werkzeug.security import generate_password_hash, check_password_hash
import sqlite3
import os

app = Flask(__name__)
app.config["SECRET_KEY"] = "change-me-in-production"  # needed for flash messages

# --------------------------------------------
# Database init (email UNIQUE, case-insensitive)
# --------------------------------------------
# Function to connect to the SQLite database and initialize the table
def init_db():
    conn = sqlite3.connect('users1.db')
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        first_name TEXT NOT NULL,
        last_name TEXT NOT NULL,
        email TEXT NOT NULL,
        password TEXT NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    ''')
    
    # Ensure case-insensitive uniqueness on email.
    # This works even if the table already exists.
    cursor.execute("""
    CREATE UNIQUE INDEX IF NOT EXISTS idx_users_email_nocase
    ON users (lower(email));
    """)
    conn.commit()
    conn.close()

# --------------------------------------------
# Routes
# --------------------------------------------
@app.route("/")
def register():
    return render_template("register.html")

@app.route("/register", methods=["POST"])
def do_register():
    first_name = request.form.get("first_name", "").strip()
    last_name  = request.form.get("last_name", "").strip()
    email      = request.form.get("email", "").strip()
    password   = request.form.get("password", "")

    # Basic validation
    if not (first_name and last_name and email and password):
        flash("All fields are required.")
        return redirect(url_for("register"))

    # Hash the password before saving
    hashed_password = generate_password_hash(password, method="pbkdf2:sha256")

    conn = sqlite3.connect('users1.db')
    cursor = conn.cursor()

    # Extra safety: pre-check for duplicates (case-insensitive)
    cursor.execute("SELECT 1 FROM users WHERE lower(email) = lower(?)", (email,))
    if cursor.fetchone():
        flash("⚠️ This email is already registered. Please use a different email.")
        conn.close()
        return redirect(url_for("register"))

    # Insert user; IntegrityError covers any race condition on the unique index
    try:
        cursor.execute(
            "INSERT INTO users (first_name, last_name, email, password) VALUES (?, ?, ?, ?)",
            (first_name, last_name, email, hashed_password)
        )
        conn.commit()
        flash("✅ Registration successful!")
    except sqlite3.IntegrityError:
        flash("⚠️ This email is already registered. Please use a different email.")
    finally:
        conn.close()

    return redirect(url_for("users"))

@app.route("/users")
def users():
    conn = sqlite3.connect('users1.db')
    cursor = conn.cursor()
    cursor.execute("SELECT first_name, last_name, email, created_at FROM users ORDER BY created_at DESC")
    rows = cursor.fetchall()
    conn.close()
    return render_template("users.html", users=rows)

# --------------------------------------------
# Entry
# --------------------------------------------
if __name__ == "__main__":
    init_db()
    app.run(port=5000, debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [29/Oct/2025 14:11:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2025 14:11:47] "GET /users HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2025 14:11:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2025 14:11:55] "POST /register HTTP/1.1" 302 -
127.0.0.1 - - [29/Oct/2025 14:11:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2025 14:11:57] "GET /users HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2025 14:11:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Oct/2025 14:12:18] "GET /users HTTP/1.1" 200 -
